# Simplified Bepro Network Model
## Introduction to Bepro

>Bepro Network is a decentralized marketplace and system that connects developers with operators and anyone looking to build open-source development repositories. The network is self-managed protocol, managed by the BEPRO token, a utility ERC20 token that enables token holders to manage disputes in the Network and earn token rewards by providing value.
https://docs.bepro.network/

There are eight mechanisms in which an agent can interact with an instance of BePro Network:

1. `createBounty`: open issues on the platform in exchange for ERC20 Tokens to developers;
2. `cancelBounty`: cancel the issue that was first created. If the bounty is canceled after the draft time period, the operation comes with a penalty of X% of the bounty;
3. `createPullRequest`: developers notify the operators they are solving the issue published;  
4. `cancelPullRequest`: developers notify the operators they are no longer solving the issue;
5. `markPullRequestAsReady`: developers notify the operators they solved the problem and are for the merge proposals;
6. `createBountyProposal`: any council, holder of 25M $BEPRO, can make a merge proposal for a resolution mechanism for a bounty. In case the same proposal is passed through the curation proccess, the council member receives X% of the bounty;
7. `disputeProposal`: council member contests a merge proposal with a new one;
8. `refuseBountyProposal`: bounty creator can refuse the bounty proposals (what is his power regarding this?);
9. `closeBounty`: proccess that takes place after the rewards are distributed to every agent accountable (including the treasury) and the bounty NFT is minted;


Fees
1. `Network Fee`: (5% currently): of the total bounty prize that goes to a treasury address for each bounty created on the system;
2. `Curator Fee` (3% currently): of the total bounty prize that goes to the Curator member that proposes the Mergee Distribution/Request;
3. `Cancel Fee` (1% currently): of the total bounty prize that goes to a treasury address when a bounty is canceled;
4. Study the possibility of new parameters and dynamic changes to the existing ones;

## Limitations and simplifications of this model
* **Bounties are either created or, canceled or closed**

* **Usage of the white-label by an operator doesn't require staking $BEPRO**

* **All bounties will be payed in $BEPRO** instead of any ERC20 tokens

* **Gas fees are counted as 0 or irrelevant for the network agents**;

* **User _behavior_ has not been modeled**. User actions are derived from the history of events of the Bepro instance being analyzed;

* **One timestep in the simulation will be equal to 24 hours**

* **Pull requests and merge proccess are abstracted**, i.e., every time a bounty is closed, the model assumes the proccess of dispute and curation has been made successfuly

---

## MODEL

### Initial conditions
These are the initial conditions of the [BEPRO Development DAO](https://development.bepro.network/)

In [405]:
# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

# cadCAD global simulation configuration list
from cadCAD import configs

# For analytics
import numpy as np
import pandas as pd

# For visualization
import plotly.express as px

# For calculus
import math
import random

# For networks
import networkx as nx

In [406]:
G = nx.MultiDiGraph()

# Initial number of agents
NUMBER_AGENTS = 100

# Max / Min balance for each agent
MAX_AGENT_BALANCE = 100000000
MIN_AGENT_BALANCE = 10000

# Generates Agent objects
for agent in range(NUMBER_AGENTS):
    # Assumes every balance within 10k and 100M $BEPRO has the same probability 
    agent_balance = int(round(MIN_AGENT_BALANCE + np.random.rand() * (MAX_AGENT_BALANCE - MIN_AGENT_BALANCE),-3))
    pCreate = random.uniform(0.1,0.2) # Probability to create a bounty
    pCancel = random.uniform(0.005, 0.01)
    pClose = random.uniform(0.01,0.05)
    bountiesCreated=[]
    bountiesCanceled=[]
    bountiesClosed=[]
    
    # Add agent to the graph
    G.add_node(
        agent,
        agentN=agent,
        agent_balance=agent_balance, 
        pCreate=pCreate, 
        pClose = pClose,
        pCancel = pCancel,
        bountiesCreated=bountiesCreated,
        bountiesCanceled=bountiesCanceled, 
        bountiesClosed=bountiesClosed
        )

# Generets bounty objects
class Bounty:
    def __init__ (
        self, 
        bountyOwner,
        reward=int(round(random.uniform(100000,2000000),-3)),
        timeBeforeClosing = int(round(random.uniform(1,60),0)),
        pCreate= random.uniform(0.05,0.25),
        isLive=True
        ):
        self.reward=reward
        self.timeBeforeClosing = timeBeforeClosing # Not used yet
        self.pCreate=pCreate
        self.bountyOwner = bountyOwner
        self.isLive=isLive
    
    def __repr__ (self):
        return 'This bounty was created by agent {}, provides reward of {} $BEPRO and will be closed within {} days.'.format(
        self.bountyOwner.get("agentN"),
        self.reward,
        self.timeBeforeClosing
        )

# Initial bounties in the network
bountiesActive=[]
bountiesCanceled=[]
bountiesClosed=[]
for i in range(NUMBER_AGENTS):
    aux = random.random()
    if aux<G.nodes[i].get('pCreate'):
        bounty=Bounty(G.nodes[i])
        G.nodes[i].get('bountiesCreated').append(bounty)
        bountiesActive.append(bounty)

beproInNetwork=0
for i in bountiesActive:
    beproInNetwork+=i.reward

In [407]:
initial_state={
    # Aux variables
    'bountiesActive':bountiesActive,
    'bountiesCanceled': bountiesCanceled,
    'bountiesClosed': bountiesClosed,
    
    # Nodes / Agents => ABM relationships
    'agentsInNetwork': G,
    
    # Analysis Variables
    'numberBountiesActive': len(bountiesActive),
    'numberBountiesCanceled': len(bountiesCanceled),
    'numberBountiesClosed': len(bountiesClosed),
    'numberAgentsInNetwork': len(G),
    'beproInNetwork': beproInNetwork,
    'feeRevenue': 0
}


In [408]:
system_params={
    'networkFee': [0.05],
    'cancelFee': [0.01],
}

print(int(round(random.random()*5,0)))

1


In [409]:
# POLICY FUNCTIONS
def p_createBounty(params, substep, state_history, previous_state):
    G = previous_state['agentsInNetwork']
    deltaNetwork = 0
    created=[]
    
    for i in range(len(G)):
        aux1 = random.random()
        if aux1<G.nodes[i].get('pCreate'):
            bounty=Bounty(G.nodes[i])
            created.append(bounty)
            deltaNetwork+=bounty.reward

    return {
        'newBountiesActive': created,
        'addBeproNetwork': deltaNetwork, 
        }
    
def p_cancel_or_closeBounty(params, substep, state_history, previous_state):
    G = previous_state['agentsInNetwork']
    
    cancelFee = params['cancelFee']
    closeFee = params['networkFee']
    
    closed=[]
    canceled=[]
    
    deltaNetwork = 0
    deltaTreasury = 0

    for i in range(len(G)):
        if len(G.nodes[i].get('bountiesCreated')) == 0:
            continue
        else:
            aux1 = random.random()
            if aux1<G.nodes[i].get('pCancel'):
                auxBounty = G.nodes[i].get('bountiesCreated')[0]
                auxBounty.isLive = False
                
                canceled.append(auxBounty)
                
                reward = auxBounty.reward
                deltaTreasury += int(round(cancelFee*reward,0))
                deltaNetwork -= reward  
                continue 
            else:
                aux2 = random.random()
                if aux2<G.nodes[i].get('pClose'):
                    auxBounty = G.nodes[i].get('bountiesCreated')[0]
                    auxBounty.isLive = False
                    
                    closed.append(auxBounty)
                    reward = auxBounty.reward
                    deltaNetwork-= reward
                    deltaTreasury += int(round(closeFee*reward,0))     
                    
    return {
        'subtracteBeproNetwork': deltaNetwork, 
        'deltaTreasuryCancelClose':deltaTreasury ,
        'canceledBounties': canceled,
        'closedBounties': closed
        }
    


# STATE UPDATE FUNCTIONS

def s_bountiesActive(params, substep, state_history, previous_state, policy_input): # This Function is not operating yet, useless for now
    bountiesActive = previous_state['bountiesActive']
    
    deltaActiveBounties = policy_input['newBountiesActive']
    deltaCanceledBounties = policy_input['canceledBounties']
    deltaClosedBounties = policy_input['closedBounties']
    
    
    for i in deltaCanceledBounties:
        bountiesActive.remove(i)
    
    for i in deltaClosedBounties:
        bountiesActive.remove(i) 
    
    for i in deltaActiveBounties:
        bountiesActive.append(i)

    return 'bountiesActive', bountiesActive

def s_bountiesCanceled(params, substep, state_history, previous_state, policy_input):
    bountiesCanceled = previous_state['bountiesCanceled']
    deltaCanceledBounties = policy_input['canceledBounties']
    
    for i in deltaCanceledBounties:
        bountiesCanceled.append(i)

    return 'bountiesCanceled', bountiesCanceled

def s_bountiesClosed(params, substep, state_history, previous_state, policy_input):
    bountiesClosed = previous_state['bountiesClosed']
    deltaClosedBounties = policy_input['closedBounties']
    
    for i in deltaClosedBounties:
        bountiesClosed.append(i)

    return 'bountiesClosed', bountiesClosed


def s_beproInNetwork(params, substep, state_history, previous_state, policy_input):
    beproInNetwork=previous_state['beproInNetwork']
    subtracteBeproNetwork = policy_input['subtracteBeproNetwork']
    addBeproNetwork = policy_input['addBeproNetwork']
    
    newTotalBepro = beproInNetwork + subtracteBeproNetwork+ addBeproNetwork
    
    return 'beproInNetwork', max(0,newTotalBepro)
    
def s_feeRevenue(params, substep, state_history, previous_state, policy_input):
    feeRevenue=previous_state['feeRevenue']
    deltaTreasury = policy_input['deltaTreasuryCancelClose']
    
    newFeeRevenue = feeRevenue + deltaTreasury
    
    return 'feeRevenue', max(0,newFeeRevenue)

def s_agentsInNetwork(params, substep, state_history, previous_state, policy_input):
    G = previous_state['agentsInNetwork']
    
    newBounties = policy_input['newBountiesActive']
    deltaCanceledBounties = policy_input['canceledBounties']
    deltaClosedBounties= policy_input['closedBounties']
    enterNodes = int(round(random.random()*5,0)) # Max people enter per timestep is 5
    
    for i in range(enterNodes):
        # Assumes every balance within 10k and 100M $BEPRO has the same probability 
        agent_balance = int(round(MIN_AGENT_BALANCE + np.random.rand() * (MAX_AGENT_BALANCE - MIN_AGENT_BALANCE),-3))
        pCreate = random.uniform(0.1,0.2) # Probability to create a bounty
        pCancel = random.uniform(0.005, 0.01)
        pClose = random.uniform(0.01,0.05)
        bountiesCreated=[]
        bountiesCanceled=[]
        bountiesClosed=[]
    
        # Add agent to the graph
        G.add_node(
            agent,
            agentN=agent,
            agent_balance=agent_balance, 
            pCreate=pCreate, 
            pClose = pClose,
            pCancel = pCancel,
            bountiesCreated=bountiesCreated,
            bountiesCanceled=bountiesCanceled, 
            bountiesClosed=bountiesClosed
            )
    
    for i in newBounties:
        numberAgent = i.bountyOwner.get('agentN')
        G.nodes[numberAgent].get('bountiesCreated').append(i)
    
    # Error being thrown, I will debug later
    # for i in deltaCanceledBounties:
    #     numberAgent = i.bountyOwner.get('agentN')
    #     G.nodes[numberAgent].get('bountiesCreated').remove(i)
    #     G.nodes[numberAgent].get('bountiesCanceled').append(i)
    
    # for i in deltaClosedBounties:
    #     numberAgent = i.bountyOwner.get('agentN')
    #     G.nodes[numberAgent].get('bountiesCreated').remove(i)
    #     G.nodes[numberAgent].get('bountiesClosed').append(i)
    return 'agentsInNetwork', G

def s_numberBountiesActive(params, substep, state_history, previous_state, policy_input):
    bountiesActive = previous_state['bountiesActive']
    return 'numberBountiesActive', max(0,len(bountiesActive))

def s_numberBountiesCanceled(params, substep, state_history, previous_state, policy_input):
    bountiesCanceled = previous_state['bountiesCanceled']
    return 'numberBountiesCanceled', max(0,len(bountiesCanceled))
    
def s_numberBountiesClosed(params, substep, state_history, previous_state, policy_input):
    bountiesClosed = previous_state['bountiesClosed']
    return 'numberBountiesClosed', max(0,len(bountiesClosed))

def s_numberAgentsInNetwork(params, substep, state_history, previous_state, policy_input):
    G = previous_state['agentsInNetwork']
    return 'numberAgentsInNetworkd', max(0,len(G))

In [410]:
partial_state_update_blocks = [ # 2 PSUBS
    { # First PSUB
        'policies':{
            'newBountiesActive': p_createBounty,
            'addBeproNetwork':  p_createBounty,
            
            'subtracteBeproNetwork': p_cancel_or_closeBounty, 
            'deltaTreasury': p_cancel_or_closeBounty,
            'closedBounties': p_cancel_or_closeBounty,
            'canceledBounties': p_cancel_or_closeBounty,
        },
        'variables': {
            'bountiesCanceled': s_bountiesCanceled,
            'bountiesClosed': s_bountiesClosed,
            'beproInNetwork': s_beproInNetwork,
            'feeRevenue': s_feeRevenue,
            'agentsInNetwork': s_agentsInNetwork  
        }
    },
    { # Second PSUB
        'policies':{},
        'variables': {
            'numberBountiesActive': s_numberBountiesActive,
            'numberBountiesCanceled': s_numberBountiesCanceled,
            'numberBountiesClosed': s_numberBountiesClosed,
            'numberAgentsInNetwork': s_numberAgentsInNetwork
        }
    }     
]

In [411]:
sim_config = config_sim({
    "N": 5,
    "T": range(20), 
    "M": system_params 
})

del configs[:]

In [412]:
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

In [404]:
exec_context = ExecutionContext()
simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (10, 2, 5, 10)
Execution Method: local_simulations
SimIDs   : [0, 0, 0, 0, 0]
SubsetIDs: [0, 0, 0, 0, 0]
Ns       : [0, 1, 2, 3, 4]
ExpIDs   : [0, 0, 0, 0, 0]
Execution Mode: parallelized
Total execution time: 1.35s


In [413]:
# Convert raw results to a Pandas DataFrame
df = pd.DataFrame(raw_result)

# Insert cadCAD parameters for each configuration into DataFrame
for config in configs:
    # Get parameters from configuration
    parameters = config.sim_config['M']
    # Get subset index from configuration
    subset_index = config.subset_id
    
    # For each parameter key value pair
    for (key, value) in parameters.items():
        # Select all DataFrame indices where subset == subset_index
        dataframe_indices = df.eval(f'subset == {subset_index}')
        # Assign each parameter key value pair to the DataFrame for the corresponding subset
        df.loc[dataframe_indices, key] = value
df

,bountiesActive,bountiesCanceled,bountiesClosed,agentsInNetwork,numberBountiesActive,numberBountiesCanceled,numberBountiesClosed,numberAgentsInNetwork,beproInNetwork,feeRevenue,simulation,subset,run,substep,timestep,numberAgentsInNetworkd,networkFee,cancelFee
0,"[This bounty was created by agent 0, provides ...",[],[],"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",23,0,0,100,21712000,0,0,0,1,0,0,NaN,0.05,0.01
1,"[This bounty was created by agent 0, provides ...",[],"[This bounty was created by agent 55, provides...","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",23,0,0,100,44368000,188800,0,0,1,1,1,NaN,0.05,0.01
2,"[This bounty was created by agent 0, provides ...",[],"[This bounty was created by agent 55, provides...","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",23,0,4,100,44368000,188800,0,0,1,2,1,100.0,0.05,0.01
3,"[This bounty was created by agent 0, provides ...","[This bounty was created by agent 74, provides...","[This bounty was created by agent 55, provides...","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",23,0,4,100,75520000,396480,0,0,1,1,2,100.0,0.05,0.01
4,"[This bounty was created by agent 0, provides ...","[This bounty was created by agent 74, provides...","[This bounty was created by agent 55, provides...","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",23,2,8,100,75520000,396480,0,0,1,2,2,100.0,0.05,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,"[This bounty was created by agent 0, provides ...","[This bounty was created by agent 22, provides...","[This bounty was created by agent 22, provides...","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",23,16,57,100,183136000,2841440,0,0,5,2,8,100.0,0.05,0.01
101,"[This bounty was created by agent 0, provides ...","[This bounty was created by agent 22, provides...","[This bounty was created by agent 22, provides...","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",23,16,57,100,196352000,3304000,0,0,5,1,9,100.0,0.05,0.01
102,"[This bounty was created by agent 0, provides ...","[This bounty was created by agent 22, provides...","[This bounty was created by agent 22, provides...","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",23,20,66,100,196352000,3304000,0,0,5,2,9,100.0,0.05,0.01
103,"[This bounty was created by agent 0, provides ...","[This bounty was created by agent 22, provides...","[This bounty was created by agent 22, provides...","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",23,20,66,100,201072000,3908160,0,0,5,1,10,100.0,0.05,0.01


In [416]:
px.line(
    df,
    x='timestep', 
    y=['numberBountiesActive', 'numberBountiesClosed'], 
    height=800 
)

In [388]:
px.line(
    df,
    x='timestep',
    y='numberAgentsInNetwork',
    color='run', 
    height=800,
)

In [414]:
px.line(
    df,
    x='timestep',
    y='feeRevenue', 
    color='run', 
    height=800,
)

# Positive variation for treasury

In [415]:
px.line(
    df,
    x='timestep',
    y='beproInNetwork', 
    color='run', 
    height=800,
)